<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
Brief explanation on our Booter database analysis methodology: <br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

# Analysis of: vaporizedbooter
#### File originally retrieved from: http://pastebin.com/ccfdEF2p

## * Libraries that I use to analyse data

In [1]:
import pandas as pd
import numpy as np

from io import StringIO
import re

from IPython.display import display,clear_output

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 0: Reading an input Booter database file<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [2]:
dumpfile='vaporizedbooter.sql'

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 1: Adaptation to our Booter database schema<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

## 1. Reading the first 'N' (100) lines of the input Booter database file

In [3]:
lines_to_read=100

with open(dumpfile) as myfile:
    firstlines=myfile.readlines()[0:lines_to_read] #put here the interval you want
    for x in firstlines:
        print(x.strip())

# Dumped by c999Shell.SQL v. 1.0 pre-release build #16

# Home page: http://ccteam.ru

#

# Host settings:

# MySQL version: (5.1.62-cll) running on 109.163.231.194 (109.163.231.194)

# Date: 03.05.2012 01:44:28

# DB: "vaporize_bootme4"

#---------------------------------------------------------

DROP TABLE IF EXISTS `email_templates`;
CREATE TABLE `email_templates` (
`id` int(5) NOT NULL AUTO_INCREMENT,
`name` varchar(200) NOT NULL,
`subject` varchar(255) NOT NULL,
`help` text,
`body` text,
PRIMARY KEY (`id`)
) ENGINE=MyISAM AUTO_INCREMENT=15 DEFAULT CHARSET=utf8;

INSERT INTO `email_templates`(`id`, `name`, `subject`, `help`, `body`) VALUES ('1', 'Registration Email', 'Please verify your email', 'This template is used to send Registration Verification Email, when Configuration->Registration Verification is set to YES', '&lt;div align=&quot;center&quot;&gt;
&lt;table cellspacing=&quot;5&quot; cellpadding=&quot;5&quot; border=&quot;0&quot; width=&quot;600&quot; style=&quot;background:

## 2. Listing tables that have content inserted into the dump file

In [4]:
def list_tables_with_insert(dumpfile):
    tables = []
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            if line.lower().startswith('insert'):
                table = re.findall(r'`(.*?)`', line)
                tables.append(table[0])
    return tables

In [5]:
tables=enumerate(list_tables_with_insert(dumpfile))

for i, item in tables:
    print(i+1,item)

1 email_templates
2 email_templates
3 email_templates
4 email_templates
5 email_templates
6 email_templates
7 email_templates
8 email_templates
9 email_templates
10 email_templates
11 email_templates
12 email_templates
13 email_templates
14 gateways
15 gateways
16 getshells
17 getshells
18 getshells
19 getshells
20 getshells
21 getshells
22 getshells
23 getshells
24 getshells
25 getshells
26 getshells
27 getshells
28 getshells
29 getshells
30 getshells
31 getshells
32 getshells
33 getshells
34 getshells
35 getshells
36 getshells
37 getshells
38 getshells
39 getshells
40 getshells
41 getshells
42 getshells
43 getshells
44 getshells
45 getshells
46 getshells
47 getshells
48 getshells
49 getshells
50 getshells
51 getshells
52 getshells
53 getshells
54 getshells
55 getshells
56 getshells
57 getshells
58 getshells
59 getshells
60 getshells
61 getshells
62 getshells
63 getshells
64 getshells
65 getshells
66 getshells
67 getshells
68 getshells
69 getshells
70 getshells
71 getshells
72 getshel

## How the list above looks like? 

<h2 align='center'>======================================================================
If NOT well-formed SQL dump file then you must first do the following:
======================================================================</h2>

## 3. Removing lines that are not part of the actual content to be analysed

## 4. Splitting and naming tables and columns

<h2 align='center'>======================================================================
Additional functions
======================================================================</h2>

## * Function to read tables from wel-formed SQL database dump

In [35]:
# For well formed SQL database dumps!
def read_inserted_table(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    already_header = False
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
                if already_header:
                    continue
            if fast_forward:
                continue
            
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
            print(data)
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                fast_forward = True
                already_header = True
    sio.seek(0)
    return sio

In [40]:
# For well formed SQL database dumps!
def read_inserted_table_entirein1line(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    already_header = False
    with open(dumpfile, 'rb') as f:
        for line in f:
            print(line)
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
                if already_header:
                    continue
            if fast_forward:
                continue
            
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert operation)
            print(data)
            #Main difference btw this fuction and the one before is this loop
            for record in data:
                try:
                    newline = record.strip('()') #Step2:remove parenthesis
                    newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                    newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),','', newline) #Step4: remove commas from the content of columns
                    newline=newline.replace('\'','') #Step5: remove single quotes
                    newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                    sio.write(newline)
                    sio.write("\n")
                except IndexError:
                    pass
                
            if line.endswith(';'):
                fast_forward = True
                already_header = True
    sio.seek(0)
    return sio

In [41]:
tablename='email_templates'

pd.read_csv(read_inserted_table_entirein1line(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(20)

b'# Dumped by c999Shell.SQL v. 1.0 pre-release build #16\n'
b'\n'
b'# Home page: http://ccteam.ru\n'
b'\n'
b'#\n'
b'\n'
b'# Host settings:\n'
b'\n'
b'# MySQL version: (5.1.62-cll) running on 109.163.231.194 (109.163.231.194)\n'
b'\n'
b'# Date: 03.05.2012 01:44:28\n'
b'\n'
b'# DB: "vaporize_bootme4"\n'
b'\n'
b'#---------------------------------------------------------\n'
b'\n'
b'DROP TABLE IF EXISTS `email_templates`;\n'
b'CREATE TABLE `email_templates` (\n'
b'  `id` int(5) NOT NULL AUTO_INCREMENT,\n'
b'  `name` varchar(200) NOT NULL,\n'
b'  `subject` varchar(255) NOT NULL,\n'
b'  `help` text,\n'
b'  `body` text,\n'
b'  PRIMARY KEY (`id`)\n'
b') ENGINE=MyISAM AUTO_INCREMENT=15 DEFAULT CHARSET=utf8;\n'
b'\n'
b"INSERT INTO `email_templates`(`id`, `name`, `subject`, `help`, `body`) VALUES ('1', 'Registration Email', 'Please verify your email', 'This template is used to send Registration Verification Email, when Configuration->Registration Verification is set to YES', '&lt;div align=&quot;c

,244,244.1,244.2
0,102,102,102
1,244,244,244
2,102,102,102
3,244,244,244
4,102,102,102
5,244,244,244
6,102,102,102
7,244,244,244
8,102,102,102
9,244,244,244


## * Converter functions for formatting content of columns
Attention, it is better to perform the conversion based on a series than a string (http://stackoverflow.com/questions/42462906/pandas-read-csv-converters-performance-issue)

In [22]:
def tolowercase(series):
    return series.str.lower()

In [23]:
def strip_singlequote(series):
    return series.str.strip('\'')

In [24]:
def timestamp2datetime(series):
    return  pd.to_datetime(series,unit='s',errors='coerce')

In [25]:
def datetimestring2datetime(series):
    return  pd.to_datetime(series, format='%d-%m-%Y %H:%M',errors='coerce')  

In [26]:
def datetimeinvertedstring2datetime(series):
    return pd.to_datetime(series, format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [27]:
def datetimeampm2datetime(series):
    return  pd.to_datetime(series,format='%d-%m-%Y %I:%M:%S %p',errors='coerce')

In [28]:
def datetimemdyampm2datetime(series):
    return  pd.to_datetime(series, format='%m-%d-%Y %I:%M:%S %p',errors='coerce') 

In [29]:
def strip_parenthesisandsemicolon(series):
    return series.str.strip(');')

In [30]:
def strip_insertintologs(series):
    return int(series.str.strip('INSERT INTO `logs` VALUES ('))

In [31]:
def strip_tab(series):
    return series.str.strip('\t')

In [32]:
def friendsenemies_type(series):
    return series.str.replace('f','friend').replace('e','enemy')
    

<h2 align='center'>======================================================================
Adapting EACH existing table
======================================================================</h2>

## 6.1. Table: 'email_templates'

### How this table looks like without modification?

In [34]:
tablename='email_templates'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(20)

,244,244.1,244.2
0,244,244,244
1,244,244,244
2,244,244,244
3,244,244,244
4,244,244,244
5,244,244,244
6,244,244,244
7,244,244,244
8,244,244,244
9,244,244,244


#### Q1: Is this table different from other previous loaded table? YES
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: blacklist
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o ID -> id
        o IP -> ip
    - Add required columns:
        o 
    - Split columns:
        o 

### Modifying table: 'blacklist'

In [25]:
# Loading the correct table name
df_blacklist = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing column names
df_blacklist.rename(inplace=True, columns={'ID':'id',
                                          'IP':'ip'})

# Adding missing columns (with "" [for future string] or np.nan [for future float])

###Showing some lines after adapt the table
df_blacklist.head(10)

,id,ip,note
0,41,127.0.0.1,test
1,48,127.0.0.1,Server
2,49,5.135.145.220,frmodz
3,40,199.16.130.194,PlayModz
4,50,188.138.106.110,test
5,51,1.1.1.1,BlackList
6,52,83.156.176.10,NaN


## 6.2. Table: 'gateway'

### How this table looks like without modification?

In [26]:
tablename='gateway'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,email
0,vps-serveur@outlook.fr


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: gateways
#### Q3: Are there modifications required? 
    - On the table name: 
        o gateway -> gateways
    - On the column type: 
        o  
    - On the column name:
        o 
    - Add required columns:
        o
    - Split columns:
        o 

### Modifying table: 'gateway'

In [27]:
# Loading the correct table name
df_gateways = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing column names

# Adding missing columns (with "" [for future string] or np.nan [for future float])

###Showing some lines after adapt the table
df_gateways.head(10)

,email
0,vps-serveur@outlook.fr


In [24]:
# Loading the correct table name
df_gateways = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing column names

# Showing some lines after adapt the table
df_gateways.head(10)

,email
0,solace@love.com


## 6.3. Table: 'iplogs'

### How this table looks like without modification?

In [28]:
tablename='iplogs'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,userID,IP,time
0,1,4267,50.17.69.56,1379189986
1,2,4267,108.171.179.20,1379189986
2,3,4161,173.252.112.114,1379190000
3,4,777,87.98.151.104,1379203403
4,5,3478,173.252.100.116,1379209015
5,6,777,202.169.236.195,1379215442
6,7,777,66.249.66.20,1379226995
7,8,777,66.249.66.20,1379240913
8,9,3983,90.3.159.111,1379278795
9,10,777,46.118.113.33,1379290796


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: logins
#### Q3: Are there modifications required? 
    - On the table name: 
        o iplogs -> logins
    - On the column type: 
        o time -> timestamp2datetime()
    - On the column name:
        o ID -> id
        o userID -> userid
        o IP -> userip
        
    - Add required columns:
        o username 
   - Split columns:
        o 

### Modifying table: 'iplogs'

In [30]:
# Loading the correct table name
df_logins = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns
df_logins['date']=timestamp2datetime(df_logins['time'])

# Changing column names
df_logins.rename(inplace=True, columns={'ID':'id',
                                        'userID':'userid',
                                        'IP':'userip'})
# Adding missing columns
df_logins['username']=np.nan

# Showing some lines after adapt the table
df_logins.head(10)

,id,userid,userip,time,date,username
0,1,4267,50.17.69.56,1379189986,2013-09-14 20:19:46,NaN
1,2,4267,108.171.179.20,1379189986,2013-09-14 20:19:46,NaN
2,3,4161,173.252.112.114,1379190000,2013-09-14 20:20:00,NaN
3,4,777,87.98.151.104,1379203403,2013-09-15 00:03:23,NaN
4,5,3478,173.252.100.116,1379209015,2013-09-15 01:36:55,NaN
5,6,777,202.169.236.195,1379215442,2013-09-15 03:24:02,NaN
6,7,777,66.249.66.20,1379226995,2013-09-15 06:36:35,NaN
7,8,777,66.249.66.20,1379240913,2013-09-15 10:28:33,NaN
8,9,3983,90.3.159.111,1379278795,2013-09-15 20:59:55,NaN
9,10,777,46.118.113.33,1379290796,2013-09-16 00:19:56,NaN


## 6.4. Table: 'logs'!!!!!

### How this table looks like without modification?

In [31]:
tablename='logs'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

b'Skipping line 57: expected 4 fields, saw 6\nSkipping line 58: expected 4 fields, saw 6\nSkipping line 59: expected 4 fields, saw 6\nSkipping line 60: expected 4 fields, saw 6\nSkipping line 61: expected 4 fields, saw 6\nSkipping line 62: expected 4 fields, saw 6\nSkipping line 63: expected 4 fields, saw 6\nSkipping line 64: expected 4 fields, saw 6\nSkipping line 65: expected 4 fields, saw 6\nSkipping line 66: expected 4 fields, saw 6\nSkipping line 67: expected 4 fields, saw 6\nSkipping line 68: expected 4 fields, saw 6\nSkipping line 69: expected 4 fields, saw 6\nSkipping line 70: expected 4 fields, saw 6\nSkipping line 71: expected 4 fields, saw 6\nSkipping line 72: expected 4 fields, saw 6\nSkipping line 73: expected 4 fields, saw 6\nSkipping line 74: expected 4 fields, saw 6\nSkipping line 75: expected 4 fields, saw 6\nSkipping line 76: expected 4 fields, saw 6\nSkipping line 77: expected 4 fields, saw 6\nSkipping line 78: expected 4 fields, saw 6\nSkipping line 79: expected 4 f

,ID,userID,IP,time
0,1,4267,50.17.69.56,1379189986
1,2,4267,108.171.179.20,1379189986
2,3,4161,173.252.112.114,1379190000
3,4,777,87.98.151.104,1379203403
4,5,3478,173.252.100.116,1379209015
5,6,777,202.169.236.195,1379215442
6,7,777,66.249.66.20,1379226995
7,8,777,66.249.66.20,1379240913
8,9,3983,90.3.159.111,1379278795
9,10,777,46.118.113.33,1379290796


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: logins
#### Q3: Are there modifications required? 
    - On the table name: 
        o iplogs -> logins
    - On the column type: 
        o date -> timestamp2datetime()
    - On the column name:
        o ID -> id
        o userID -> userid
        o logged -> userip
        
    - Add required columns:
        o username 
   - Split columns:
        o 

### Modifying table: 'logs'

In [28]:
# Loading the correct table name
df_logins = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns
df_logins['date']=timestamp2datetime(df_logins['date'])

# Changing column names
df_logins.rename(inplace=True, columns={'ID':'id',
                                        'userID':'userid',
                                        'logged':'userip'})
# Adding missing columns
df_logins['username']=np.nan

# Showing some lines after adapt the table
df_logins.head(10)

,id,userid,userip,date,username
0,110,169,99.228.120.154,2012-08-01 12:47:46,NaN
1,3,17,82.19.49.66,2012-07-26 19:21:16,NaN
2,643,123,66.249.85.58,2012-12-12 22:42:02,NaN
3,109,169,97.112.157.117,2012-08-01 12:43:55,NaN
4,108,169,98.127.42.196,2012-08-01 12:41:09,NaN
5,12,62,90.184.187.69,2012-07-28 12:00:25,NaN
6,13,62,62.107.45.66,2012-07-28 13:24:20,NaN
7,14,62,90.184.187.69,2012-07-28 18:59:18,NaN
8,15,62,90.184.187.69,2012-07-28 18:59:37,NaN
9,16,62,62.107.45.66,2012-07-28 19:18:27,NaN


## 6.5. Table: 'months'

### How this table looks like without modification?

In [32]:
tablename='months'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,name,description,mbt,unit,length,price
0,108,Ultimate Pack 2,5600,5600,Months,1,53
1,107,MemberShip Pack,3600,3600,Months,1,38
2,106,Ultimate Pack,Pack,2400,Days,1,32
3,105,Blue Pack,1800,1800,Months,1,26
4,104,Red Pack,1200,1200,Months,1,18
5,103,Yellow Pack,900,900,Months,1,12
6,102,Green Pack,600,600,Months,1,6
7,101,Silver Pack,120,300,Months,1,3


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: plans
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o ID -> planid
        o name -> planname
        o description -> plandescr
        o mbt -> maxboottime
        
    - Add required columns:
        o concurrecy
    - Split columns:
        o 

### Modifying table: 'months'

In [35]:
# Loading the correct table name
df_plans = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing names of columns
df_plans.rename(inplace=True, columns={'ID':'planid',
                                      ' name': 'planname',
                                      ' description': ' plandescr',
                                      ' mbt':' maxboottime'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_plans['concurrency']=np.nan

# Showing some lines after adapt the table
df_plans.head(10)

,planid,name,description,mbt,unit,length,price,concurrency
0,108,Ultimate Pack 2,5600,5600,Months,1,53,NaN
1,107,MemberShip Pack,3600,3600,Months,1,38,NaN
2,106,Ultimate Pack,Pack,2400,Days,1,32,NaN
3,105,Blue Pack,1800,1800,Months,1,26,NaN
4,104,Red Pack,1200,1200,Months,1,18,NaN
5,103,Yellow Pack,900,900,Months,1,12,NaN
6,102,Green Pack,600,600,Months,1,6,NaN
7,101,Silver Pack,120,300,Months,1,3,NaN


## 6.6. Table: 'news'

### How this table looks like without modification?

In [36]:
tablename='news'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,title,detail,date
0,37,NS,Hello NationalStresser is now the rotation ser...,1379588490


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

## 6.7. Table: 'payments'

### How this table looks like without modification?

In [37]:
tablename='payments'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,paid,plan,user,email,tid,date
0,1,15,31,194,kevin.benaroche@hotmail.fr,11W69868WX2888303,1354389662
1,2,3,38,497,mikedelija@gmail.com,9B578750YB669902P,1354421172
2,3,3,38,498,lisburbage@blueyonder.co.uk,6X266797TH430374M,1354466613
3,4,3,38,213,boolastar2@hotmail.fr,4SP87033JK2354245,1354481462
4,5,15,31,502,sunilmeboss1@yahoo.com,3RV205149P7488819,1354485493
5,6,30,33,503,youngminecrafter@live.com,7456208460098734Y,1354486827
6,7,15,31,482,hassell.w@gmail.com,05373516BM892900W,1354487180
7,8,3,38,500,oleandreas.98@hotmail.com,6XS1683572882372T,1354488792
8,9,3,38,513,michaelflamini6155@hotmail.fr,4FM57049D9988614Y,1354515469
9,10,3,38,518,mark3526200@gmail.com,5X405403U9429834F,1354556304


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES 
    - This table looks like: payments
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o date -> timestamp2datetime()
    - On the column name:
        o ID -> id
        o paid -> amountpaid
        o plan ->planid
        o user -> userid
        o email -> paymentemail
        
    - Add required columns:
        o username
    - Split columns:
        o 

### Modifying table: 'payments'

In [38]:
# Loading the correct table name
df_payments = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns
df_payments['date'] = timestamp2datetime(df_payments['date']) 

# Changing names of columns
df_payments.rename(inplace=True, columns={'ID':'id',
                                        'paid':'amount',
                                         'plan':'planid',
                                         'user':'userid',
                                         'email':'paymentemail'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_payments['username']=np.nan

# Showing some lines after adapt the table
df_payments.head(10)

,id,amount,planid,userid,paymentemail,tid,date,username
0,1,15,31,194,kevin.benaroche@hotmail.fr,11W69868WX2888303,2012-12-01 19:21:02,NaN
1,2,3,38,497,mikedelija@gmail.com,9B578750YB669902P,2012-12-02 04:06:12,NaN
2,3,3,38,498,lisburbage@blueyonder.co.uk,6X266797TH430374M,2012-12-02 16:43:33,NaN
3,4,3,38,213,boolastar2@hotmail.fr,4SP87033JK2354245,2012-12-02 20:51:02,NaN
4,5,15,31,502,sunilmeboss1@yahoo.com,3RV205149P7488819,2012-12-02 21:58:13,NaN
5,6,30,33,503,youngminecrafter@live.com,7456208460098734Y,2012-12-02 22:20:27,NaN
6,7,15,31,482,hassell.w@gmail.com,05373516BM892900W,2012-12-02 22:26:20,NaN
7,8,3,38,500,oleandreas.98@hotmail.com,6XS1683572882372T,2012-12-02 22:53:12,NaN
8,9,3,38,513,michaelflamini6155@hotmail.fr,4FM57049D9988614Y,2012-12-03 06:17:49,NaN
9,10,3,38,518,mark3526200@gmail.com,5X405403U9429834F,2012-12-03 17:38:24,NaN


## 6.8. Table: 'plans'

### How this table looks like without modification?

In [39]:
tablename='plans'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,name,description,mbt,unit,length,price
0,84,Blue Pack,MemberShip Blue Pack,1200,Years,10,26
1,85,Red Pack,MemberShip New Pack Unlocked Restricted Area,1800,Years,10,32
2,86,Ultimate Pack,Ultimate Pack MemberShip,5600,Years,30,98
3,87,Pack Premium,NS,2400,Years,10,38
4,81,Gold Pack,MemberShip Restricted Unlocked,300,Years,10,6
5,82,Green Pack,MemberShip NationalStresser Buy Now,600,Years,10,12
6,83,Yellow Pack,MemberShip Buisness Yellow Pack,900,Years,10,18
7,89,NS,4200,3600,Years,10,57


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES
    - This table looks like: plans
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o ID -> planid
        o name -> planname
        o description -> plandescr
        o mbt -> maxboottime
    - Add required columns:
        o concurrecy
    - Split columns:
        o 

### Modifying table: 'plans'

In [40]:
# Loading the correct table name
df_plans = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns

# Changing names of columns
df_plans.rename(inplace=True, columns={'ID':'planid',
                                         'name':'planname',
                                         'description': 'plandescr',
                                         'mbt':'maxboottime'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_plans['concurrency']=np.nan

# Showing some lines after adapt the table
df_plans.head(10)

,planid,planname,plandescr,maxboottime,unit,length,price,concurrency
0,84,Blue Pack,MemberShip Blue Pack,1200,Years,10,26,NaN
1,85,Red Pack,MemberShip New Pack Unlocked Restricted Area,1800,Years,10,32,NaN
2,86,Ultimate Pack,Ultimate Pack MemberShip,5600,Years,30,98,NaN
3,87,Pack Premium,NS,2400,Years,10,38,NaN
4,81,Gold Pack,MemberShip Restricted Unlocked,300,Years,10,6,NaN
5,82,Green Pack,MemberShip NationalStresser Buy Now,600,Years,10,12,NaN
6,83,Yellow Pack,MemberShip Buisness Yellow Pack,900,Years,10,18,NaN
7,89,NS,4200,3600,Years,10,57,NaN


## 6.9. Table: 'settings'

### How this table looks like without modification?

In [41]:
tablename='settings'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,API


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? NO
    - This table looks like: 
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o 
    - On the column name:
        o 
    - Add required columns:
        o 
    - Split columns:
        o 

## 6.9. Table: 'users'

### How this table looks like without modification?

In [42]:
tablename='users'

pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False).head(10)

,ID,username,password,status,rank,mbt,expire
0,42,kyubi,946ae17537269629c017992d68496cc9981ff88195b3fa...,0,0,0,1367691180
1,43,Azapo,c3555b15c17ec6d775561b03261c94ea910f83eff525f5...,0,0,0,1367777580
2,45,iExoTicK,ba3253876aed6bc22d4a6ff53d8406c6ad864195ed144a...,0,0,0,1590789600
3,47,Pikachu7390,3b443808c21868ab5c21102572dbb1e86ff641db12715d...,0,0,600,1432936800
4,48,Akerio,3fce4c8c3ee963f1ad71c4bb393ff09b3fb9a21935cbe2...,0,0,0,1369937580
5,49,Hacks116,9f281740d815393fba449c824665ffc357bb1e2c39f506...,0,0,0,1369937580
6,50,chybe21,b109f3bbbc244eb82441917ed06d618b9008dd09b3befd...,0,0,0,1369937580
7,51,Daveskee,9c7bf89a08dbb0bcf38b6d759476c0d785b49a1010870a...,0,0,0,1369937580
8,52,FOXNONAD,418d940c10e17197ebc4007af1d8e555c6f8d81bc0c060...,0,0,1200,1451516400
9,53,gamingzmod,ba3253876aed6bc22d4a6ff53d8406c6ad864195ed144a...,0,0,300,1590789600


#### Q1: Is this table different from other previous loaded table? YES 
#### Q2: Is this table similar to any table in the generic Booter database schema? YES 
    - This table looks like: users
#### Q3: Are there modifications required? 
    - On the table name: 
        o 
    - On the column type: 
        o expire -> timestamp2datetime()
    - On the column name:
        o ID -> userid
              
    - Add required columns:
        o useremail 
        o planid
    - Split columns:
        o 

### Modifying table: 'users'

In [43]:
# Loading the correct table name
df_users = pd.read_csv(read_inserted_table(dumpfile, tablename),delimiter=",",error_bad_lines=False)

# Converting columns
df_users['expire'] = timestamp2datetime(df_users['expire']) 

# Changing names of columns
df_users.rename(inplace=True, columns={'ID':'userid'})

# Creating empty columns (with "" [for future string] or np.nan [for future float])
df_users[' useremail']=np.nan
df_users['planid']=np.nan

# Showing some lines after adapt the table
df_users.head(10)

,userid,username,password,status,rank,mbt,expire,useremail,planid
0,42,kyubi,946ae17537269629c017992d68496cc9981ff88195b3fa...,0,0,0,2013-05-04 18:13:00,NaN,NaN
1,43,Azapo,c3555b15c17ec6d775561b03261c94ea910f83eff525f5...,0,0,0,2013-05-05 18:13:00,NaN,NaN
2,45,iExoTicK,ba3253876aed6bc22d4a6ff53d8406c6ad864195ed144a...,0,0,0,2020-05-29 22:00:00,NaN,NaN
3,47,Pikachu7390,3b443808c21868ab5c21102572dbb1e86ff641db12715d...,0,0,600,2015-05-29 22:00:00,NaN,NaN
4,48,Akerio,3fce4c8c3ee963f1ad71c4bb393ff09b3fb9a21935cbe2...,0,0,0,2013-05-30 18:13:00,NaN,NaN
5,49,Hacks116,9f281740d815393fba449c824665ffc357bb1e2c39f506...,0,0,0,2013-05-30 18:13:00,NaN,NaN
6,50,chybe21,b109f3bbbc244eb82441917ed06d618b9008dd09b3befd...,0,0,0,2013-05-30 18:13:00,NaN,NaN
7,51,Daveskee,9c7bf89a08dbb0bcf38b6d759476c0d785b49a1010870a...,0,0,0,2013-05-30 18:13:00,NaN,NaN
8,52,FOXNONAD,418d940c10e17197ebc4007af1d8e555c6f8d81bc0c060...,0,0,1200,2015-12-30 23:00:00,NaN,NaN
9,53,gamingzmod,ba3253876aed6bc22d4a6ff53d8406c6ad864195ed144a...,0,0,300,2020-05-29 22:00:00,NaN,NaN


In [48]:
len(df_users), len(df_users.drop_duplicates())

(1892, 1892)

<h2 align='center'>======================================================================
<br>Final step of the manual part
======================================================================</h2>

## 7. Adding missing tables accordingly to our generic Booter database 

In [ ]:
df_attacks= pd.DataFrame(columns=['id','userid','username','targetip','targeturl','duration','port','type','date'])
#df_blacklist=pd.DataFrame(columns=['id','ip','note'])
#df_friendsenemies=pd.DataFrame(columns=['id','ip','note','userid','type'])
#df_gateways=pd.DataFrame(columns=['email'])
#df_logins=pd.DataFrame(columns=['id','userid','username','userip','date'])
#df_payments=pd.DataFrame(columns=['id','userid','username','amountpaid','paymentemail','planid','tid','date'])
df_plans=pd.DataFrame(columns=['planid','planname','plandescr','price','maxboottime','concurrency'])
df_servers=pd.DataFrame(columns=['id','ip'])
df_settings=pd.DataFrame(columns=['url','sitename','siteemail'])
#df_users=pd.DataFrame(columns=['userid','username','useremail','password','expire','plan'])      
df_webshells=pd.DataFrame(columns=['id','url','status','lastchecked','attacktype'])

<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 2: Data Enrichment<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

### * Depending of the size of the data, this part can take HOURS. I tested for both small and big datasets and it worked. Be pacient. This will pay-off!

## * Libraries needed to retrieve information from external databases

In [ ]:
import subprocess
import cfscrape
from lxml import etree
import os.path
import random
import time 

## * Function to enrich IP addresseswith AS information and country 

In [ ]:
# THANKS TO: team-cymru.org
def iptoasn(iplist_teamcymruformat_filelocation,outputfile):
    cat = subprocess.Popen(['cat', iplist_teamcymruformat_filelocation], 
                            stdout=subprocess.PIPE)
    
    netcat = subprocess.Popen(['netcat', 'whois.cymru.com', '43'],
                              stdin=cat.stdout,
                              stdout=outputfile)
    time.sleep(3) #for some reason the poll does not work! This was the way to overcome the waiting time.
    
    return netcat.stdout      

## * Function to check if an IP address was Tor node in a given moment

In [ ]:
# THANKS TO: exonerator.torproject.org
def WasTorNode(ip, date ):
    url="https://exonerator.torproject.org/?ip="+ip+"&timestamp="+date
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content    
    html_tree = etree.HTML(scraped_html)
    result=html_tree.xpath("//h3[@class='panel-title']/text()") # I was looking for <h3 class="panel-title">Result is positive</h3>
    tor_node=True if result == ['Result is positive'] else False
    return tor_node 
# 'date' MUST BE formated as: Year-month-day (2016-03-31)

## * Discovering the middle date of the database

In [ ]:
try:
    try:
        middle_date=(min(df_attacks['date'])+((max(df_attacks['date'])-min(df_attacks['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_logins['date'])+((max(df_logins['date'])-min(df_logins['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_payments['date'])+((max(df_payments['date'])-min(df_payments['date']))/2))
        raise
    except:
        pass
except Exception:
    print("There is no date in the entire dataset")

date_tor_check = middle_date.strftime('%Y-%m-%d')
date_iptoasn_lookup= str(middle_date)
print(date_tor_check)

## * Preparing to Perform IP to ASN info

In [ ]:
df_logins['middledate']=date_iptoasn_lookup
df_attacks['middledate']=date_iptoasn_lookup
df_friendsenemies['middledate']=date_iptoasn_lookup
df_blacklist['middledate']=date_iptoasn_lookup

## 2.1.1  Lookup IP to ASN info of table: logins

In [ ]:
if (os.path.exists('enrichments/logins_iptoasn_out')== False):
    logins_iptoasn_in = open('enrichments/logins_iptoasn_in', 'w+')
    logins_iptoasn_in.write('begin\nverbose\n')
    df_logins[['userip','middledate']].drop_duplicates().to_csv(logins_iptoasn_in,header=False,index=False,sep="\t") 
    logins_iptoasn_in.write('end')
    logins_iptoasn_in.close()

    logins_iptoasn_out = open('logins_iptoasn_out', 'w+')
    iptoasn('logins_iptoasn_in',logins_iptoasn_out)
    logins_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_logins_iptoasn = pd.read_csv('enrichments/logins_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

df_logins_extended= pd.merge(df_logins,
                              df_logins_iptoasn,
                              how = 'left',
                              left_on = 'userip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_logins_extended.rename(columns={'asn':'srcasn', 
                                   'ip':'srcip', 
                                   'bgp_prefix':'srcbgp_prefix', 
                                   'country':'srccountry' ,
                                   'registry':'srcregistry',
                                   'info_date':'srcinfo_date',
                                   'info_request':'srcinfo_request'},
                         inplace=True)


df_logins_extended.head(5)

## 2.1.2  Lookup IP to ASN info of table: attacks

In [ ]:
if (os.path.exists('enrichments/attacks_iptoasn_out')== False):
    attacks_iptoasn_in = open('enrichments/attacks_iptoasn_in', 'w+')
    attacks_iptoasn_in.write('begin\nverbose\n')
    df_attacks[['targetip','middledate']].drop_duplicates().to_csv(attacks_iptoasn_in,header=False,index=False,sep="\t") 
    attacks_iptoasn_in.write('end')
    attacks_iptoasn_in.close()

    attacks_iptoasn_out = open('attacks_iptoasn_out', 'w+')
    iptoasn('attacks_iptoasn_in',attacks_iptoasn_out)
    attacks_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_attacks_iptoasn = pd.read_csv('enrichments/attacks_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_attacks_extended= pd.merge(df_attacks,
                              df_attacks_iptoasn,
                              how = 'left',
                              left_on = 'targetip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_attacks_extended.rename(columns={'asn':'targetasn', 
                                   'ip_y':'targetip', 
                                   'bgp_prefix':'targetbgp_prefix', 
                                   'country_y':'targetcountry' ,
                                   'registry':'targetregistry',
                                   'info_date':'targetinfo_date',
                                   'info_request':'targetinfo_request'},
                         inplace=True)
df_attacks_extended.head(5)

## 2.1.3  Lookup IP to ASN info of table: friendsenemies

In [ ]:
if (os.path.exists('enrichments/friendsenemies_iptoasn_out')== False):
    friendsenemies_iptoasn_in = open('enrichments/friendsenemies_iptoasn_in', 'w+')
    friendsenemies_iptoasn_in.write('begin\nverbose\n')
    df_friendsenemies[['ip','middledate']].drop_duplicates().to_csv(friendsenemies_iptoasn_in,header=False,index=False,sep="\t") 
    friendsenemies_iptoasn_in.write('end')
    friendsenemies_iptoasn_in.close()

    friendsenemies_iptoasn_out = open('friendsenemies_iptoasn_out', 'w+')
    iptoasn('friendsenemies_iptoasn_in',friendsenemies_iptoasn_out)
    friendsenemies_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_friendsenemies_iptoasn = pd.read_csv('enrichments/friendsenemies_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_friendsenemies_extended= pd.merge(df_friendsenemies,
                              df_friendsenemies_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_friendsenemies_extended.rename(columns={'asn':'friendsenemiesasn', 
                                   'ip':'friendsenemiesip', 
                                   'bgp_prefix':'friendsenemiesbgp_prefix', 
                                   'country':'friendsenemiescountry' ,
                                   'registry':'friendsenemiesregistry',
                                   'info_date':'friendsenemiesinfo_date',
                                   'info_request':'friendsenemiesinfo_request',
                                   'as_name': 'friendsenemiesas_name'},
                         inplace=True)

## 2.1.4  Lookup IP to ASN info of table: blacklist

In [ ]:
if (os.path.exists('enrichments/blacklist_iptoasn_out')== False):
    blacklist_iptoasn_in = open('enrichments/blacklist_iptoasn_in', 'w+')
    blacklist_iptoasn_in.write('begin\nverbose\n')
    df_blacklist[['ip','middledate']].drop_duplicates().to_csv(blacklist_iptoasn_in,header=False,index=False,sep="\t") 
    blacklist_iptoasn_in.write('end')
    blacklist_iptoasn_in.close()

    blacklist_iptoasn_out = open('blacklist_iptoasn_out', 'w+')
    iptoasn('blacklist_iptoasn_in',blacklist_iptoasn_out)
    blacklist_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_blacklist_iptoasn = pd.read_csv('enrichments/blacklist_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])

# Merging the iptoasn with the queried column
df_blacklist_extended= pd.merge(df_blacklist,
                              df_blacklist_iptoasn,
                              how = 'left',
                              left_on = 'ip',
                              right_on = 'ip')

# Changing name of columns to avoid misunderstandings
df_blacklist_extended.rename(columns={'asn':'blacklistasn', 
                                   'ip':'blacklistip', 
                                   'bgp_prefix':'blacklistbgp_prefix', 
                                   'country':'blacklistcountry' ,
                                   'registry':'blacklistregistry',
                                   'info_date':'blacklistinfo_date',
                                   'info_request':'blacklistinfo_request',
                                   'as_name': 'blacklistas_name'},
                         inplace=True)
df_blacklist_extended.head(5)

## 2.2.1. Check if IP was a TOR node for table: login

In [ ]:
date_tor_check

In [ ]:
if len(df_logins['userip'].unique())<1200:
    if (os.path.exists('enrichments/logins_torcheck')== False):
        print("Note: it can take a while to finish...")
        logins_torcheck = open('logins_torcheck', 'w+')
        for i in df_logins['userip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=logins_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            logins_torcheck.flush()

        logins_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.")
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_logins_torcheck = pd.read_csv('enrichments/logins_torcheck',\
                          delimiter="\s+",\
                          names = ['userip', 'tor'])

## 2.2.2. Check if IP was a TOR node for table: attacks

In [ ]:
if len(df_attacks['targetip'].unique())<1200:
    if (os.path.exists('enrichments/attacks_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_attacks['targetip'].unique())*3,"seconds (in the worst case).")

        attacks_torcheck = open('attacks_torcheck', 'w+')

        for i in df_attacks['targetip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=attacks_torcheck)
            print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            attacks_torcheck.flush()

        attacks_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_attacks_torcheck = pd.read_csv('enrichments/attacks_torcheck',\
                          delimiter="\s+",\
                          names = ['targetip', 'tor'])

## 2.2.3. Check if IP was a TOR node for table: friendsenemies

In [ ]:
if len(df_friendsenemies['ip'].unique()) <1200:
    if (os.path.exists('enrichments/friendsenemies_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_friendsenemies['ip'].unique())*3,"seconds (in the worst case).")

        friendsenemies_torcheck = open('enrichments/friendsenemies_torcheck', 'w+')

        for i in df_friendsenemies['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=friendsenemies_torcheck)
        #     print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            friendsenemies_torcheck.flush()

        friendsenemies_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_friendsenemies_torcheck = pd.read_csv('enrichments/friendsenemies_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## 2.2.4. Check if IP was a TOR node for table: blacklist

In [ ]:
if len(df_blacklist['ip'].unique()) < 1200:
    if (os.path.exists('enrichments/blacklist_torcheck')== False):
        print("Note: it can take a while to finish...",len(df_blacklist['ip'].unique())*3,"seconds (in the worst case).")

        blacklist_torcheck = open('enrichments/blacklist_torcheck', 'w+')

        for i in df_blacklist['ip'].unique():
            wasTor=WasTorNode(i,date_tor_check)
            print(i, wasTor, file=blacklist_torcheck)
    #         print(i, wasTor) #DEBUGING =D
            time.sleep(random.randint(1,3)) #adding some random sleep time
            blacklist_torcheck.flush()

        blacklist_torcheck.close()
    else:
        print("The IP addresses from this table were already checked.") 
else:
        print("Aborted!!! It will take more than one hour to analyse!")

In [ ]:
df_blacklist_torcheck = pd.read_csv('enrichments/blacklist_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

## * Relation between Attack dates and Login dates

In [ ]:
def nearestDate(base_date, date_list):
    nearest={}
    for date in date_list:
        if (base_date.timestamp() - date.timestamp())>=0:
            nearest[base_date.timestamp() - date.timestamp()]= date
    return nearest[min(nearest.keys())]

In [ ]:
#This is the TOTAL number records to be checks!!!!
len(df_attacks)

In [ ]:
df_attacks['nearestlogin']=""
df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])
    
if len(df_attacks)>0 and len(df_logins)>0:
    #When was the last login of the user that performed attacks
    df_attacks['nearestlogin']=""
    df_attacks['nearestlogin']=pd.to_datetime(df_attacks['nearestlogin'])

    for index, row in df_attacks.head(100).iterrows():
        intermediate_df= df_logins[df_logins['username']==row['username']]
        nearestlogindate= nearestDate(row['date'],intermediate_df['date'])
        df_attacks.set_value(index, 'nearestlogin', nearestlogindate)

        #DEBUGGING
        if index % 1000 == 0:
            print(index,": +1000 records analysed")

In [ ]:
if len(df_attacks['nearestlogin'].value_counts()) >1:
    df_attacks_and_logins = pd.merge(df_attacks_extended,
                                     df_logins_extended,
                                     how = 'left',
                                     left_on = ['username','nearestlogin'],
                                     right_on = ['username','date'])
else:
    df_attacks_and_logins=""

<h2 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
STEP 3: Automatic Analysis
<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h3> 

## * Libraries that I use to plot figs

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import *
%matplotlib inline

import seaborn as sns
import itertools

plt.style.use('seaborn-muted')
# plt.rcParams['font.family'] = 'serif'
# plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

## A.1. Number of records per table (part of the generic Booter database schema)

In [ ]:
len(df_users),len(df_logins),len(df_attacks),len(df_payments),len(df_settings),len(df_gateways), len(df_friendsenemies),len(df_blacklist),len(df_webshells),len(df_servers),len(df_plans)

## A.2. Number of users, customers, attackers, and their intersection

In [ ]:
if len(set(df_users['userid'].unique())) > 1:
    users_set=set(df_users['userid'].unique())
else:
    users_set=set(df_users['username'].unique())
    
if len(set(df_logins['userid'].unique())) > 1:
    userslogin_set=set(df_logins['userid'].unique())
else:
    userslogin_set=set(df_logins['username'].unique())

if len(set(df_payments['userid'].unique())) > 1:
    customers_set=set(df_payments['userid'].unique())
else:
    customers_set=set(df_payments['username'].unique())

if len(set(df_attacks['userid'].unique())) > 1:
    attackers_set=set(df_attacks['userid'].unique())
else:
    attackers_set=set(df_attacks['username'].unique())

intersec_customers_attacker=pd.Series(list(customers_set.intersection(attackers_set)))
intersec_users_customers=pd.Series(list(users_set.intersection(customers_set)))
intersec_users_attackers=pd.Series(list(users_set.intersection(attackers_set)))
intersec_users_customers_attackers=pd.Series(list(users_set.intersection(customers_set).intersection(attackers_set)))

In [ ]:
len(users_set),len(userslogin_set),len(customers_set),len(attackers_set)

In [ ]:
fig = plt.figure(figsize=(4,3))
fig.suptitle('Users, Customers & Attackers', fontsize=14)

ax = plt.subplot2grid((1,1), (0,0))

venn=venn3(ax=ax,subsets = {'001':len(attackers_set)-len(intersec_customers_attacker)-len(intersec_users_attackers)+len(intersec_users_customers_attackers), 
                            '010':len(customers_set)-len(intersec_users_customers)-len(intersec_customers_attacker)+len(intersec_users_customers_attackers), 
                            '011':len(intersec_customers_attacker)-len(intersec_users_customers_attackers),
                            '100':len(users_set)-len(intersec_users_customers)-len(intersec_users_attackers)+len(intersec_users_customers_attackers),
                            '101':len(intersec_users_attackers)-len(intersec_users_customers_attackers),
                            '110':len(intersec_users_customers)-len(intersec_users_customers_attackers),
                            '111':len(intersec_users_customers_attackers)},\
          set_labels = ('Users', 'Customers','Attackers'),\
          alpha=1)
try:
    venn.get_patch_by_id('100').set_color('#404096')
except:
    print("*Users set is empty!")  
    
try:
    venn.get_patch_by_id('110').set_color('#DEA73A')
except:
    print("*Customers set is empty!")   

try:
    venn.get_patch_by_id('001').set_color('#D92120')
except:
    print("*Attackers set is empty!")

fig.show()
# fig.savefig('../figs/timeseries_attacks.eps', format='eps', dpi=1200) 

## A.3. Distribution of login times per user

In [ ]:
if len(df_logins['userid'].value_counts()) > 0:
    num_distinct_logins_per_user=df_logins['userid'].value_counts()
else:
    num_distinct_logins_per_user=df_logins['username'].value_counts()

freq_distinct_logins_per_user=num_distinct_logins_per_user.value_counts()
cum_dist_user_logins = np.linspace(0.,1.,len(num_distinct_logins_per_user))
cdf_user_logins = pd.Series(cum_dist_user_logins, index=num_distinct_logins_per_user.sort_values())

In [ ]:
if len(num_distinct_logins_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Login Times by Users:', fontsize=14, y=1.05,x=0.35)
    
    #Plot CDF
    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_logins.plot(ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# logins")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    ax1.set_title("")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_logins_per_user.plot(ax=ax2,kind='pie', 
                                       labels=None, 
                                       legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    
    
    
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_logins_per_user.values)/(freq_distinct_logins_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_logins_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/login_times.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.3. Number of Users that Login via TOR

In [ ]:
len(df_logins_torcheck[df_logins_torcheck['tor']==True]['userip'].value_counts())

## 5.4. Number of Distinct IP addresses by Users

In [ ]:
if len(df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()) >0:
    num_distinct_ips_per_user=df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()
else:
    num_distinct_ips_per_user=df_logins.groupby(['username','userip']).size().reset_index()['username'].value_counts()
    
freq_distinct_ips_per_user=num_distinct_ips_per_user.value_counts()
cum_dist_user_ips = np.linspace(0.,1.,len(num_distinct_ips_per_user))
cdf_user_ips = pd.Series(cum_dist_user_ips, index=num_distinct_ips_per_user.sort_values())

In [ ]:
if len(num_distinct_ips_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Distinct IP address used by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_ips.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_ips_per_user.plot(ax=ax2,kind='pie',
                                    labels=None,legend=False,
                                       startangle=270,
#                                        colors=sns.color_palette()
                                       )
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_ips_per_user.values)/(freq_distinct_ips_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_ips_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.5, 1.),
               fontsize=10)
    ####
    
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/num_ips_by_users.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.5. Number of Payments by Users

In [ ]:
if len(df_payments['userid'].value_counts())>0:
    num_distinct_payments_per_user=df_payments['userid'].value_counts()
else:
    num_distinct_payments_per_user=df_payments['username'].value_counts()

freq_distinct_payments_per_user=num_distinct_payments_per_user.value_counts()
cum_dist_user_payments = np.linspace(0.,1.,len(num_distinct_payments_per_user))
cdf_user_payments = pd.Series(cum_dist_user_payments, index=num_distinct_payments_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Number of Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# Payment")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_per_user.plot(ax=ax2,kind='pie', 
                                         labels=None,legend=False,
                                         startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_per_user.values)/(freq_distinct_payments_per_user.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/payments_distribution.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.6. Total Amount of Money Earned

In [ ]:
if len(df_payments['amountpaid']) >0:
    total_earned=df_payments['amountpaid'].values.sum()
    'US$ {:,.2f}'.format(float(total_earned))
else:
    print("Unfortunately, there is no data available!")

## 5.7. Amount of Money Paid by Users

In [ ]:
num_distinct_payments_money_per_user=df_payments['amountpaid'].value_counts()
freq_distinct_payments_money_per_user=num_distinct_payments_money_per_user.value_counts()
cum_dist_user_payments_money = np.linspace(0.,1.,len(num_distinct_payments_money_per_user))
cdf_user_payments_money = pd.Series(cum_dist_user_payments_money, index=num_distinct_payments_money_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_money_per_user)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Distribution of Money Payments by Users:', fontsize=14, y=1.05, x=0.4)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments_money.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("$")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_money_per_user.plot(ax=ax2,kind='pie', 
                                               labels=None,legend=False,
                                               startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_distinct_payments_money_per_user.values)/(freq_distinct_payments_money_per_user.values.sum())
    labels = ['${0:1.2f} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_distinct_payments_money_per_user.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.6, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/distribution_amount_paid.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")


## 5.8. Countries from where Users Access Booters

In [ ]:
logins_country_distribution_sorted = df_logins_iptoasn['country'].value_counts(sort=True,ascending=True)
logins_country_distribution = df_logins_iptoasn['country'].value_counts()

In [ ]:
if len(logins_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Users Accessing from Countries:', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    logins_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    logins_country_distribution.plot(ax=ax2,kind='pie', 
                                     labels=None,legend=False,
                                     startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*logins_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(logins_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/user_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.9. Countries of Blacklisted IPs

In [ ]:
blacklist_country_distribution=df_blacklist_iptoasn['country'].value_counts()
blacklist_country_distribution_sorted=df_blacklist_iptoasn['country'].value_counts(sort=True,ascending=True)

In [ ]:
if len(blacklist_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of blacklisted IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    blacklist_country_distribution_sorted.plot(ax=ax1,kind='barh')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    blacklist_country_distribution.plot(ax=ax2,kind='pie', 
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*blacklist_country_distribution.values)/(logins_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(blacklist_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/blacklist_countries.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.10. Countries of Target IPs

In [ ]:
attacks_country_distribution=df_attacks_iptoasn['country'].value_counts()

In [ ]:
if len(attacks_country_distribution)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Countries of target IPs', fontsize=14, y=1.05, x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    attacks_country_distribution.plot(ax=ax1,kind='bar')
    ax1.set_ylabel("# Access")
    ax1.set_xlabel("Country")
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    attacks_country_distribution.plot(ax=ax2,kind='pie', 
                                      labels=None,legend=False,
                                      startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*attacks_country_distribution.values)/(attacks_country_distribution.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(attacks_country_distribution.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()
    fig.show()
    fig.savefig('figs/attack_countries_distribution.eps',bbox_inches='tight', format='eps', dpi=1200) 
else:
    print("Unfortunately, there is no data available!")

## 5.12. Attacks on Same Target

In [ ]:
num_attacks_on_sametarget=df_attacks['targetip'].value_counts()

freq_num_attacks_on_sametarget=num_attacks_on_sametarget.value_counts()
cum_num_attacks_on_sametarget = np.linspace(0.,1.,len(num_attacks_on_sametarget))
cdf_num_attacks_on_sametarget = pd.Series(cum_num_attacks_on_sametarget, index=num_attacks_on_sametarget.sort_values())

In [ ]:
if len(num_attacks_on_sametarget)>0:
    fig = plt.figure(figsize=(6, 3))
    fig.suptitle('* Attacks on Same Targets:', fontsize=14, y=1.05,x=0.28)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_num_attacks_on_sametarget.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_num_attacks_on_sametarget.plot(ax=ax2,kind='pie',
                                        labels=None,legend=False,
                                        startangle=270)
    ax2.set_ylabel("")
    ####
    # For Pie Chart Better Legend box
    porcent = (100.*freq_num_attacks_on_sametarget.values)/(freq_num_attacks_on_sametarget.values.sum())
    labels = ['{0} - {1:1.2f} %'.\
              format(i,j) for i,j in zip(freq_num_attacks_on_sametarget.index, porcent)]
    # To Sort Legend (defaulf: keeps the same order)
#     patches, labels, dummy =  zip(*sorted(zip(patches, labels, freq_distinct_logins_per_user.values),
#                                           key=lambda x: x[2],
#                                           reverse=True))
    legend_show_top=10
    ax2.legend(ax2.patches[0:legend_show_top], 
               labels[0:legend_show_top], 
               bbox_to_anchor=(1.55, 1.),
               fontsize=10)
    ####
    fig.tight_layout()

    fig.show()
    fig.savefig('figs/attacks_on_same_target.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

## 5.11. Attacks per day (timeseries) 

In [ ]:
if len(df_attacks)>0:
    attack_timeseries=df_attacks.set_index(['date']).groupby(pd.TimeGrouper(freq='D')).agg(['count'])['action']
    attack_mean_perday=attack_timeseries.mean()
    attack_median_perday=attack_timeseries.median()
else:
    attack_timeseries=""

In [ ]:
if len(attack_timeseries)>0:
    fig = plt.figure(figsize=(6,3))

    ax1 = plt.subplot2grid((1,1), (0,0), rowspan=2)
    attack_timeseries.plot(ax=ax1,
                           lw=2,
                           legend=False,
    #                        style='--'
                          )

    # X and Y Labels and Ticks
    ax1.set_xlabel("Time (bin=day)")
    ax1.set_ylabel("# Attacks")

    ax1.annotate(str(int(attack_median_perday[0]))+' (median)', 
                 (min(df_attacks['date']), attack_median_perday),
                 xytext=(350, -1), 
                 textcoords='offset points',
                 color='black', 
                 arrowprops=dict(arrowstyle='-|>',
                                 color='black'))
    fig.savefig('figs/attacks_timeseries.eps', bbox_inches='tight',format='eps', dpi=1200)

else:
    print("Unfortunately, there is no data available!")

## 5.12. Time Between Logins and Attacks

In [ ]:
if len(df_attacks_and_logins)>0:
    print("redo")
else:
    print("Unfortunately, there is no data available!")

## 5.13. Who attack whom? (users on the country level)

In [ ]:
if len(df_attacks_extended)>0 and len(df_logins_extended)>0:
    merged_attacks_logins = pd.merge(df_attacks_extended,
                                  df_logins_extended,
                                  how = 'left',
                                  left_on = 'date',
                                  right_on = 'date')[['targetcountry','srccountry']]

    who_against_whom = merged_attacks_logins.groupby(['targetcountry','srccountry'])\
                            .size()\
                            .reset_index()\
                            .pivot('srccountry','targetcountry',0)
else:
    who_against_whom =""

In [ ]:
if len(who_against_whom)>0:
    fig = plt.figure(figsize=(8,12))
    fig.suptitle('Countries of Target IPs', fontsize=14, y=.92)

    ax1 = plt.subplot2grid((1,1), (0,0))
    sns.set()
    sns.heatmap(who_against_whom,
                ax=ax1,
#                 cmap="YlGnBu",
    #             linewidths=.5,
    #             annot=True
                )

    ax1.set_ylabel("Source Country")
    ax1.set_xlabel("Attack Target Country")

    fig.show()
    fig.savefig('figs/who_attack_whom.eps', bbox_inches='tight',format='eps', dpi=1200)
else:
    print("Unfortunately, there is no data available!")

<h1 align="center">!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
FIFTH PART: Query Interface<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

In [ ]:
from ipywidgets import widgets, interact, Dropdown,HTML
from IPython.display import display,clear_output

In [ ]:
def search_by_userid_submit(sender):
    clear_output()
    print("Searching by id =",search_by_userid.value)
    
    if len(df_users[df_users['userid']== int(search_by_userid.value)])>0:
        display(df_users[df_logins['userid']== int(search_by_userid.value)])
    
    if len(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])>0:
        display(df_logins_extended[df_logins_extended['userid']== int(search_by_userid.value)])
    
    if len(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])>0:
        display(df_attacks_extended[df_attacks_extended['userid']== int(search_by_userid.value)])
    
    if len(df_payments[df_payments['userid']== int(search_by_userid.value)])>0:
        display(df_payments[df_payments['userid']== int(search_by_userid.value)])

def search_by_username_submit(sender):
    clear_output()
    print("Searching by username =",search_by_username.value,"\n")
    
    if len(df_users[df_users['username']== search_by_username.value])>0:
        print("Table df_users:")
        display(df_users[df_users['username']== search_by_username.value])
    
    if len(df_logins_extended[df_logins_extended['username']== search_by_username.value])>0:
        print("Table df_logins:")
        display(df_logins_extended[df_logins_extended['username']== search_by_username.value])

    if len(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])>0:
        print("Table df_attacks:")
        display(df_attacks_extended[df_attacks_extended['username']== search_by_username.value])
    
    if len(df_payments[df_payments['username']== search_by_username.value])>0:
        print("Table df_payments:")
        display(df_payments[df_payments['username']== search_by_username.value])
        
def search_by_ip_submit(sender):
    clear_output()
    print("Searching by IP address =",str(search_by_ip.value),"\n")
    
    if len(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])>0:
        print("Table df_logins (as ATTACKER:")
        display(df_logins_extended[df_logins_extended['userip']== str(search_by_ip.value)])
     
    if len(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetip']== str(search_by_ip.value)])  
      
       
def search_by_asn_submit(sender):
    clear_output()
    print("Searching by Autonomous System Number (ASN) =",search_by_asn.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srcasn']== int(search_by_asn.value)])
    
    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

    if len(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetasn']== str(search_by_asn.value)])

        
country_list=pd.read_csv('https://raw.githubusercontent.com/datasets/country-list/master/data.csv',delimiter=",",error_bad_lines=False)
def search_by_country_submit():
    country_code=country_list[country_list['Name']==search_by_country.value]['Code'].values[0]
    clear_output()
    print("Searching by Country =",search_by_country.value,"\n")
    
    if len(df_logins_extended[df_logins_extended['srccountry']== country_code])>0:
        print("Table df_logins (as ATTACKER):")
        display(df_logins_extended[df_logins_extended['srccountry']== country_code])
    
    if len(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])>0:
        print("Table df_attacks (as TARGET):")
        display(df_attacks_extended[df_attacks_extended['targetcountry']== country_code])
    

In [ ]:
display(HTML('<h3>*Search by (only) one field per time:</h3>'))

search_by_username = widgets.Text(description="username")
display(search_by_username)
search_by_username.on_submit(search_by_username_submit)

search_by_userid = widgets.Text(description="userid")
display(search_by_userid)
search_by_userid.on_submit(search_by_userid_submit)

search_by_ip = widgets.Text(description="IP")
display(search_by_ip)
search_by_ip.on_submit(search_by_ip_submit)

search_by_asn = widgets.Text(description="ASN")
display(search_by_asn)
search_by_asn.on_submit(search_by_asn_submit)

search_by_country = Dropdown(description="Country", options=country_list['Name'].tolist())
search_by_country.on_trait_change(search_by_country_submit, name="value")
display(search_by_country)